In [27]:
# IMPORTA LE DIPENDENZE
import yfinance as yf 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation  
from prophet.diagnostics import performance_metrics 
from datetime import datetime, timedelta
from tabulate import tabulate
import warnings

# Disabilita i warning di Prophet
warnings.filterwarnings('ignore')

# Imposta il tema dark come default
pio.templates.default = "plotly_dark"

In [28]:
# Variabili globali per la configurazione
#                                                                PARAMETRI PERSONALIZZABILI DALL'UTENTE ⭐

######################################################################################################
SYMBOL = 'BTC-USD'                                              # ⭐                         #
######################################################################################################
######################################################################################################
SELECTED_TF = '1d'                                              # ⭐ Timeframe per l'analisi      #
######################################################################################################
######################################################################################################
FORECAST_PERIODS = 20                                           # ⭐ PERIODO DI FORECAST           #   
######################################################################################################


In [29]:
# Download and data cleaned

END_DATE = datetime.now().strftime('%Y-%m-%d')
TIMEFRAMES = ['1d', '1h', '30m', '15m', '5m', '1m']

def calculate_start_date(SELECTED_TF):
    """Calcola la data di inizio in base al timeframe selezionato."""
    if SELECTED_TF == '1d':
        return '2015-01-01'  # Scarica tutto lo storico disponibile
    elif SELECTED_TF == '1h':
        return (datetime.now() - timedelta(days=729)).strftime('%Y-%m-%d')    # Ultimi 729 giorni
    elif SELECTED_TF in ['30m', '15m', '5m']:       
        return (datetime.now() - timedelta(days=59)).strftime('%Y-%m-%d')     # Ultimi 59 giorni
    elif SELECTED_TF in ['1m']: 
        return (datetime.now() - timedelta(days=8)).strftime('%Y-%m-%d')     # Ultimi 8 giorni
# Questa soluzione e' stata adottata per via dei limiti di yfinance nel download dei dati


data = yf.download(SYMBOL, start=calculate_start_date(SELECTED_TF), end=END_DATE, interval=SELECTED_TF)   


# Filtra i dati per gli ultimi 4 anni
def filter_last_4_years(data):
    """Filtra il DataFrame per includere solo gli ultimi 4 anni."""
    end_date = data.index.max()  # Data finale
    start_date = end_date - pd.DateOffset(years=4)  # Data di inizio 4 anni fa
    return data.loc[start_date:end_date]


# Filtra i dati per gli ultimi 1 anni
def filter_last_1_year(data):
    """Filtra il DataFrame per includere solo gli ultimi 1 anno."""
    end_date = data.index.max()  # Data finale
    start_date = end_date - pd.DateOffset(years=1)  # Data di inizio 1 anno fa
    return data.loc[start_date:end_date]


# Pulizia dei dati per rimuovere valori nulli e infiniti
data.replace([np.inf, -np.inf], np.nan, inplace=True)           # Sostituisce i valori infiniti positivi e negativi con NaN
data.interpolate(method='linear', inplace=True)                 # Interpola i valori NaN con un metodo lineare
null_values_after = data.isnull().sum().sum()                   # Conta i valori nulli dopo la pulizia
null_values_before = data.isnull().sum().sum()                  # Conta i valori nulli prima della pulizia
zero_values_after = (data == 0).sum().sum()                     # Conta i valori zero dopo la pulizia
zero_values_before = (data == 0).sum().sum()                    # Conta i valori zero prima della pulizia
if null_values_before > 0 or zero_values_before > 0:
    data.replace([np.inf, -np.inf], np.nan, inplace=True)           # Sostituisce i valori infiniti positivi e negativi con NaN
data.interpolate(method='linear', inplace=True)                


# Debug sul numero di valori nulli e zero prima del trattamento di pulizia, solo se presenti
if null_values_before > 0 or zero_values_before > 0:
    print(f"\nNumero di valori nulli prima del trattamento: {null_values_before}")
    print(f"\nNumero di valori zero prima del trattamento: {zero_values_before}")


# Pulizia dei dati
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.replace(0, np.nan, inplace=True)
data.dropna(inplace=True)
data.ffill(inplace=True)


# Debug sul numero di valori nulli e zero dopo il trattamento di pulizia
if null_values_after > 0 or zero_values_after > 0:
    print(f"\nNumero di valori nulli dopo il trattamento: {null_values_after}")
    print(f"\nNumero di valori zero dopo il trattamento: {zero_values_after}")


# Stampa se ci sono valori nulli o zero
if null_values_after == 0 and zero_values_after == 0:
    print("\nNon ci sono valori nulli o zero.")
else:
    print("\nCi sono valori nulli o zero.")


# Debug sul numero di records trattati dopo la pulizia
print(f"\nNumero di records dopo la pulizia: {len(data)}")
print(f"\nTimeframe usato: {SELECTED_TF}\n")
print("\nDownload Data Cleaned")


# Dati scaricati e ripuliti
print(f"\nDati scaricati con successo per {SYMBOL} da {calculate_start_date(SELECTED_TF)} a {END_DATE}.")
print(tabulate(data.head(10), headers='keys', tablefmt='grid'))
print(tabulate(data.tail(10), headers='keys', tablefmt='grid'))

# Aggiungi questa funzione per graficare i dati
def plot_data(data):
    """Grafica i dati usando Plotly con tema dark."""
    # Debug: verifica dei dati
    print("\nVerifica dati prima del plotting:")
    print(f"Numero di righe nel dataset: {len(data)}")
    print("\nPrime 5 righe dei dati:")
    print(tabulate(data.head(), headers='keys', tablefmt='grid'))
    
    # Verifica che ci siano dati validi
    if len(data) == 0:
        print("ERRORE: Il dataset è vuoto!")
        return
    
    fig = go.Figure()
    
    # Aggiungi la traccia del prezzo di chiusura
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data[('Close', 'BTC-USD')],
            name='Prezzo di Chiusura',
            line=dict(color='cyan', width=2)
        )
    )
    
    # Imposta il layout
    fig.update_layout(
        title=dict(
            text=f'Andamento di {SYMBOL} nell\'ultimo anno',
            font=dict(size=24)
        ),
        xaxis_title='Data',
        yaxis_title='Prezzo (USD)',
        template='plotly_dark',
        showlegend=True,
        width=1800,
        height=600,
        paper_bgcolor='rgba(0,0,0,0.9)',
        plot_bgcolor='rgba(0,0,0,0.9)',
        # Configurazione della legenda sotto il grafico
        legend=dict(
            orientation="h",     # orientamento orizzontale
            yanchor="bottom",   # ancora della legenda in basso
            y=-0.2,            # posizione verticale (negativa per metterla sotto il grafico)
            xanchor="center",   # ancora orizzontale al centro
            x=0.5              # posizione orizzontale al centro
        )
    )
    
    # Aggiungi griglia con colore grigio e trasparenza
    fig.update_xaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    fig.update_yaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    
    fig.show()

# Filtra i dati per l'ultimo anno e grafica
last_year_data = filter_last_1_year(data)  # Filtra i dati per l'ultimo anno
plot_data(last_year_data)  # Grafica i dati


[*********************100%***********************]  1 of 1 completed


Non ci sono valori nulli o zero.

Numero di records dopo la pulizia: 3708

Timeframe usato: 1d


Download Data Cleaned

Dati scaricati con successo per BTC-USD da 2015-01-01 a 2025-02-25.
+---------------------+------------------------+-----------------------+----------------------+-----------------------+-------------------------+
| Date                |   ('Close', 'BTC-USD') |   ('High', 'BTC-USD') |   ('Low', 'BTC-USD') |   ('Open', 'BTC-USD') |   ('Volume', 'BTC-USD') |
+=====================+========================+=======================+======================+=======================+=========================+
| 2015-01-01 00:00:00 |                314.249 |               320.435 |              314.003 |               320.435 |             8.03655e+06 |
+---------------------+------------------------+-----------------------+----------------------+-----------------------+-------------------------+
| 2015-01-02 00:00:00 |                315.032 |               315.839 |         

In [30]:
# Preparazione dataset per passarlo a Prophet
# Dobbiamo mantenere solo le colonne 'Date' e 'Close' e rinominarle in 'ds' e 'y'

# Estrai solo la colonna 'Close' per il ticker specifico
df = data[['Close']].reset_index()  # Seleziona solo la colonna 'Close' per il ticker specifico

# Rinomina le colonne, gestendo il multi-indice
df.columns = ['ds', 'y']  # Rinomina le colonne in modo appropriato

# Rimuovi il fuso orario dalla colonna 'ds'
df['ds'] = df['ds'].dt.tz_localize(None)  # Rimuove il fuso orario

# Stampa il DataFrame per verificare i dati
print("DataFrame di addestramento:")
print(tabulate(df.tail(10), headers='keys', tablefmt='grid'))


# Assicurati che 'y' contenga solo valori numerici
df['y'] = pd.to_numeric(df['y'], errors='coerce')  # Converte 'y' in numerico, sostituisce errori con NaN
df.dropna(inplace=True)  # Rimuovi eventuali righe con valori nulli

# Assicurati che 'ds' sia di tipo datetime e 'y' sia numerico
df['ds'] = pd.to_datetime(df['ds'], errors='coerce')  # Converte 'ds' in datetime, sostituisce errori con NaT
df.dropna(inplace=True)  # Rimuovi eventuali righe con valori nulli dopo la conversione

DataFrame di addestramento:
+------+---------------------+---------+
|      | ds                  |       y |
+======+=====================+=========+
| 3698 | 2025-02-15 00:00:00 | 97580.4 |
+------+---------------------+---------+
| 3699 | 2025-02-16 00:00:00 | 96175   |
+------+---------------------+---------+
| 3700 | 2025-02-17 00:00:00 | 95773.4 |
+------+---------------------+---------+
| 3701 | 2025-02-18 00:00:00 | 95539.5 |
+------+---------------------+---------+
| 3702 | 2025-02-19 00:00:00 | 96635.6 |
+------+---------------------+---------+
| 3703 | 2025-02-20 00:00:00 | 98333.9 |
+------+---------------------+---------+
| 3704 | 2025-02-21 00:00:00 | 96125.5 |
+------+---------------------+---------+
| 3705 | 2025-02-22 00:00:00 | 96577.8 |
+------+---------------------+---------+
| 3706 | 2025-02-23 00:00:00 | 96273.9 |
+------+---------------------+---------+
| 3707 | 2025-02-24 00:00:00 | 91418.2 |
+------+---------------------+---------+


In [31]:
# Calcolo FORECAST 

# Assicurati che 'ds' sia di tipo datetime e 'y' sia numerico
df['ds'] = pd.to_datetime(df['ds'])  # Converte 'ds' in datetime
df['y'] = pd.to_numeric(df['y'], errors='coerce')  # Converte 'y' in numerico, sostituisce errori con NaN

m = Prophet()
m.fit(df)

futuro = m.make_future_dataframe(FORECAST_PERIODS)

forecast = m.predict(futuro)

print("Forecast:")
print(tabulate(forecast.tail(10), headers='keys', tablefmt='grid'))


21:14:39 - cmdstanpy - INFO - Chain [1] start processing
21:14:40 - cmdstanpy - INFO - Chain [1] done processing


Forecast:
+------+---------------------+---------+--------------+--------------+---------------+---------------+------------------+------------------------+------------------------+-----------+----------------+----------------+----------+----------------+----------------+------------------------+------------------------------+------------------------------+---------+
|      | ds                  |   trend |   yhat_lower |   yhat_upper |   trend_lower |   trend_upper |   additive_terms |   additive_terms_lower |   additive_terms_upper |    weekly |   weekly_lower |   weekly_upper |   yearly |   yearly_lower |   yearly_upper |   multiplicative_terms |   multiplicative_terms_lower |   multiplicative_terms_upper |    yhat |
+======+=====================+=========+==============+==============+===============+===============+==================+========================+========================+===========+================+================+==========+================+================+========

In [32]:
# Visualizzazione storico e previsione

# Crea un grafico personalizzato
fig1 = go.Figure()

# Aggiungi i dati storici
fig1.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Prezzo Storico', line=dict(color='yellow')))  # Prezzo storico in giallo

# Aggiungi i dati di previsione
fig1.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Previsione', line=dict(color='cyan')))  # Previsione in cyan

# Aggiungi intervallo di confidenza
fig1.add_traces(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Intervallo Inferiore', line=dict(color='lightblue', dash='dash')))
fig1.add_traces(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Intervallo Superiore', line=dict(color='lightblue', dash='dash')))

# Imposta layout
fig1.update_layout(
    title='Previsione con Prezzo Storico',
    xaxis_title='Data',
    yaxis_title='Prezzo (USD)',
    template='plotly_dark',
    width=1800, 
    height=600,
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.9)',
    # Configurazione della legenda sotto il grafico
    legend=dict(
        orientation="h",     # orientamento orizzontale
        yanchor="bottom",   # ancora della legenda in basso
        y=-0.2,            # posizione verticale (negativa per metterla sotto il grafico)
        xanchor="center",   # ancora orizzontale al centro
        x=0.5              # posizione orizzontale al centro
    )
)

# Aggiungi griglia con colore grigio e trasparenza
fig1.update_xaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)  # Imposta il colore della griglia con trasparenza
fig1.update_yaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)  # Imposta il colore della griglia con trasparenza

fig1.show()

In [33]:
def plot_forecast(forecast, df):
    """Grafica i risultati del forecast con due grafici separati."""
    
    # Primo grafico - Errore di previsione
    fig1 = go.Figure()
    
    if 'y' in df.columns:
        # Allinea le dimensioni per evitare errori
        min_length = min(len(forecast), len(df))
        fig1.add_trace(
            go.Scatter(
                x=forecast['ds'][:min_length],
                y=forecast['yhat'][:min_length] - df['y'][:min_length],
                name='Errore',
                line=dict(color='red', width=2)
            )
        )
    
    # Aggiorna il layout del primo grafico
    fig1.update_layout(
        title='Errore di previsione',
        xaxis_title='Data',
        yaxis_title='Errore (USD)',
        template='plotly_dark',
        width=1800,
        height=600,
        paper_bgcolor='rgba(0,0,0,0.9)',
        plot_bgcolor='rgba(0,0,0,0.9)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )
    
    # Aggiungi linea orizzontale a zero
    fig1.add_hline(y=0, line_dash="dash", line_color="white", line_width=1, opacity=0.3)
    
    # Aggiungi griglia con colore grigio e trasparenza
    fig1.update_xaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    fig1.update_yaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    
    # Mostra il primo grafico
    fig1.show()
    
    # Secondo grafico - yhat
    fig2 = go.Figure()
    
    fig2.add_trace(
        go.Scatter(
            x=forecast['ds'],
            y=forecast['yhat'],
            name='Previsione (yhat)',
            line=dict(color='cyan', width=2)
        )
    )
    
    # Aggiorna il layout del secondo grafico
    fig2.update_layout(
        title='Previsione (yhat)',
        xaxis_title='Data',
        yaxis_title='Valore Previsto (USD)',
        template='plotly_dark',
        width=1800,
        height=600,
        paper_bgcolor='rgba(0,0,0,0.9)',
        plot_bgcolor='rgba(0,0,0,0.9)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )
    
    # Aggiungi griglia con colore grigio e trasparenza
    fig2.update_xaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    fig2.update_yaxes(showgrid=True, gridcolor='rgba(128, 128, 128, 0.1)', gridwidth=1)
    
    # Mostra il secondo grafico
    fig2.show()

# Plotta i risultati del forecast
plot_forecast(forecast, df)

In [34]:
# Verifica Pattern Stagionali
fig2 = plot_components_plotly(m, forecast)

# Aggiorna il layout con tema dark
fig2.update_layout(
    title='Verifica Pattern Stagionali',
    width=1800,
    height=1000,
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.9)'
)

# Determina il numero totale di subplot
n_rows = len(fig2.data) // 2 + (len(fig2.data) % 2)

# Applica le modifiche a tutti i subplot
for row in range(1, n_rows + 1):
    # Aggiorna la griglia per ogni subplot
    fig2.update_xaxes(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.1)',
        row=row,
        col=1
    )
    fig2.update_yaxes(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.1)',
        row=row,
        col=1
    )

# Aggiorna tutte le linee dello zero
for trace in fig2.data:
    if trace.name == 'zero':
        trace.line.color = 'rgba(255,255,255,0.2)'
        trace.line.dash = 'dash'  # Aggiunge anche uno stile tratteggiato per maggiore visibilità

fig2.show()

In [35]:
# Cross Validation
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')
print("\nNumero totale di records: ", len(df_cv))

print(tabulate(df_cv.head(5), headers='keys', tablefmt='grid'))
print(tabulate(df_cv.tail(5), headers='keys', tablefmt='grid'))


  0%|          | 0/15 [00:00<?, ?it/s]

21:14:41 - cmdstanpy - INFO - Chain [1] start processing
21:14:41 - cmdstanpy - INFO - Chain [1] done processing
21:14:41 - cmdstanpy - INFO - Chain [1] start processing
21:14:42 - cmdstanpy - INFO - Chain [1] done processing
21:14:42 - cmdstanpy - INFO - Chain [1] start processing
21:14:42 - cmdstanpy - INFO - Chain [1] done processing
21:14:42 - cmdstanpy - INFO - Chain [1] start processing
21:14:42 - cmdstanpy - INFO - Chain [1] done processing
21:14:43 - cmdstanpy - INFO - Chain [1] start processing
21:14:43 - cmdstanpy - INFO - Chain [1] done processing
21:14:43 - cmdstanpy - INFO - Chain [1] start processing
21:14:44 - cmdstanpy - INFO - Chain [1] done processing
21:14:44 - cmdstanpy - INFO - Chain [1] start processing
21:14:44 - cmdstanpy - INFO - Chain [1] done processing
21:14:45 - cmdstanpy - INFO - Chain [1] start processing
21:14:45 - cmdstanpy - INFO - Chain [1] done processing
21:14:45 - cmdstanpy - INFO - Chain [1] start processing
21:14:46 - cmdstanpy - INFO - Chain [1]


Numero totale di records:  5475
+----+---------------------+---------+--------------+--------------+---------+---------------------+
|    | ds                  |    yhat |   yhat_lower |   yhat_upper |       y | cutoff              |
+====+=====================+=========+==============+==============+=========+=====================+
|  0 | 2017-04-03 00:00:00 | 1119.91 |      1077.96 |      1165.33 | 1143.81 | 2017-04-02 00:00:00 |
+----+---------------------+---------+--------------+--------------+---------+---------------------+
|  1 | 2017-04-04 00:00:00 | 1123.83 |      1072.33 |      1169.42 | 1133.25 | 2017-04-02 00:00:00 |
+----+---------------------+---------+--------------+--------------+---------+---------------------+
|  2 | 2017-04-05 00:00:00 | 1125.88 |      1079.08 |      1171.32 | 1124.78 | 2017-04-02 00:00:00 |
+----+---------------------+---------+--------------+--------------+---------+---------------------+
|  3 | 2017-04-06 00:00:00 | 1130.69 |      1083.08 |     

In [36]:
# Stampiamo le metriche di performance
df_p = performance_metrics(df_cv)

print("\nNumero totale di records in base al periodo di cross validation: ", len(df_p))
print(tabulate(df_p, headers='keys', tablefmt='grid'))


Numero totale di records in base al periodo di cross validation:  329
+-----+-------------------+-------------+---------+----------+----------+----------+----------+------------+
|     | horizon           |         mse |    rmse |      mae |     mape |    mdape |    smape |   coverage |
+=====+===================+=============+=========+==========+==========+==========+==========+============+
|   0 | 37 days 00:00:00  | 1.72954e+08 | 13151.2 |  9003.64 | 0.364834 | 0.41225  | 0.383495 |  0.106642  |
+-----+-------------------+-------------+---------+----------+----------+----------+----------+------------+
|   1 | 38 days 00:00:00  | 1.74078e+08 | 13193.9 |  9053.69 | 0.36686  | 0.41225  | 0.386758 |  0.10128   |
+-----+-------------------+-------------+---------+----------+----------+----------+----------+------------+
|   2 | 39 days 00:00:00  | 1.75492e+08 | 13247.3 |  9109.39 | 0.369133 | 0.417212 | 0.390311 |  0.0957952 |
+-----+-------------------+-------------+---------+------

In [37]:
# Ultimo record delle Performance Metrics
df_p2 = performance_metrics(df_cv, rolling_window=1)
# Utilizzo di tabulate per mostrare tutti i records di df_p in formato tabellare
print(tabulate(df_p2, headers='keys', tablefmt='grid'))


+----+-------------------+-------------+---------+---------+----------+----------+----------+------------+
|    | horizon           |         mse |    rmse |     mae |     mape |    mdape |    smape |   coverage |
+====+===================+=============+=========+=========+==========+==========+==========+============+
|  0 | 365 days 00:00:00 | 6.97516e+08 | 26410.5 | 18771.2 | 0.868714 | 0.617941 | 0.841573 |   0.145753 |
+----+-------------------+-------------+---------+---------+----------+----------+----------+------------+


In [38]:
def plot_metrics(df_p):
    """Grafica le metriche di performance usando Plotly."""
    
    # Crea il layout dei subplot (ora 3x3 ma useremo solo 7 subplot)
    fig = make_subplots(  # noqa: F821 # type: ignore
        rows=3, cols=3,
        subplot_titles=(
            'Andamento del RMSE nel tempo', 'Andamento del MAE nel tempo', 
            'Andamento del MAPE nel tempo', 'Andamento del MDAPE nel tempo',
            'Andamento del SMAPE nel tempo', 'Andamento del Coverage nel tempo',
            'Andamento del MSE nel tempo'
        ),
        specs=[
            [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, None, None]  # Ultimo row ha solo un plot
        ]
    )

    # Definizione delle metriche e loro colori
    metrics = [
        ('rmse', 'orange', 1, 1), ('mae', 'blue', 1, 2),
        ('mape', 'green', 1, 3), ('mdape', 'red', 2, 1),
        ('smape', 'purple', 2, 2), ('coverage', 'brown', 2, 3),
        ('mse', 'cyan', 3, 1)
    ]

    # Aggiungi ogni metrica come traccia
    for metric, color, row, col in metrics:
        fig.add_trace(
            go.Scatter(
                x=df_p.index,
                y=df_p[metric],
                name=metric.upper(),
                line=dict(color=color, width=2),
                mode='lines+markers',
                marker=dict(size=6)
            ),
            row=row, col=col
        )

    # Calcola il massimo errore RMSE
    max_rmse = df_p2['rmse'].iloc[0]
    
    # Aggiungi un'annotazione elegante per il massimo errore
    fig.add_annotation(
        text=f"<b>Massimo Errore (RMSE)</b><br><span style='font-size: 28px'>${max_rmse:.2f}</span>",
        xref="paper", yref="paper",
        x=0.85, y=0.15,  # Posizione nell'area vuota a destra in basso
        showarrow=False,
        font=dict(
            family="Arial",
            size=20,
            color="darkorange"
        ),
        align="center",
        bordercolor="darkorange",
        borderwidth=2,
        borderpad=15,
        bgcolor="rgba(0,0,0,0.8)",
        opacity=0.8
    )

    # Aggiorna il layout
    fig.update_layout(
        height=1200,
        width=1800,
        showlegend=True,
        template='plotly_dark',
        paper_bgcolor='rgba(0,0,0,0.9)',
        plot_bgcolor='rgba(0,0,0,0.9)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=100, b=100)  # Aumenta i margini per l'annotazione
    )

    # Aggiorna gli assi per i subplot esistenti
    for i in range(1, 8):  # Ora solo 7 subplot
        row = (i-1)//3 + 1
        col = ((i-1)%3) + 1
        fig.update_xaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128, 128, 128, 0.1)',
            title_text="Numero di Record",
            row=row,
            col=col
        )
        fig.update_yaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128, 128, 128, 0.1)',
            row=row,
            col=col
        )

    fig.show()

# Chiamata alla funzione
plot_metrics(df_p)

RMSE (Root Mean Square Error): Misura la deviazione media al quadrato tra i valori previsti e quelli osservati. È utile per capire la precisione di un modello di previsione. Valori più bassi indicano previsioni più accurate.

MAE (Mean Absolute Error): Calcola la media degli errori assoluti tra i valori previsti e quelli osservati. È una misura semplice e intuitiva della precisione di un modello, con valori più bassi che indicano maggiore accuratezza.

MAPE (Mean Absolute Percentage Error): Esprime l'errore medio assoluto come percentuale rispetto ai valori osservati. È utile per confrontare la precisione tra diversi modelli o serie temporali.

MDAPE (Median Absolute Percentage Error): Simile al MAPE, ma utilizza la mediana invece della media. Questo rende la misura più robusta rispetto a valori anomali (outliers).

SMAPE (Symmetric Mean Absolute Percentage Error): Una variazione del MAPE che è simmetrica rispetto ai valori previsti e osservati. È utile quando si vuole dare uguale importanza alle deviazioni positive e negative.

COVERAGE: In ambito statistico e di machine learning, la coverage si riferisce alla percentuale di valori o casi che rientrano entro un determinato intervallo di confidenza. Ad esempio, un modello di previsione potrebbe generare intervalli di confidenza del 95%, e la coverage indicherebbe quanti di questi intervalli contengono effettivamente il valore vero. Una buona coverage indica che il modello è ben calibrato e che gli intervalli di confidenza sono affidabili.

MSE (Mean Squared Error): Calcola la media degli errori al quadrato tra i valori previsti e quelli osservati. È una misura molto utilizzata per valutare la precisione di un modello, specialmente quando si desidera penalizzare maggiormente gli errori più grandi. Valori di MSE più bassi indicano un modello più accurato.

Queste metriche sono tutte utilizzate per valutare la qualità delle previsioni di un modello statistico o di machine learning

In [39]:
help(Prophet)

Help on class Prophet in module prophet.forecaster:

class Prophet(builtins.object)
 |  Prophet(growth='linear', changepoints=None, n_changepoints=25, changepoint_range=0.8, yearly_seasonality='auto', weekly_seasonality='auto', daily_seasonality='auto', holidays=None, seasonality_mode='additive', seasonality_prior_scale=10.0, holidays_prior_scale=10.0, changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, uncertainty_samples=1000, stan_backend=None, scaling: str = 'absmax', holidays_mode=None)
 |
 |  Prophet forecaster.
 |
 |  Parameters
 |  ----------
 |  growth: String 'linear', 'logistic' or 'flat' to specify a linear, logistic or
 |      flat trend.
 |  changepoints: List of dates at which to include potential changepoints. If
 |      not specified, potential changepoints are selected automatically.
 |  n_changepoints: Number of potential changepoints to include. Not used
 |      if input `changepoints` is supplied. If `changepoints` is not supplied,
 |      then n_chan

In [40]:
help(Prophet.fit)

Help on function fit in module prophet.forecaster:

fit(self, df, **kwargs)
    Fit the Prophet model.

    This sets self.params to contain the fitted model parameters. It is a
    dictionary parameter names as keys and the following items:
        k (Mx1 array): M posterior samples of the initial slope.
        m (Mx1 array): The initial intercept.
        delta (MxN array): The slope change at each of N changepoints.
        beta (MxK matrix): Coefficients for K seasonality features.
        sigma_obs (Mx1 array): Noise level.
    Note that M=1 if MAP estimation.

    Parameters
    ----------
    df: pd.DataFrame containing the history. Must have columns ds (date
        type) and y, the time series. If self.growth is 'logistic', then
        df must also have a column cap that specifies the capacity at
        each ds.
    kwargs: Additional arguments passed to the optimizing or sampling
        functions in Stan.

    Returns
    -------
    The fitted Prophet object.

